# Dask Job Queue on Summit

**Emily Costa**

07/31/2019
***

This tutorial will use the Python packages, pyUSID and pyrcoscopy, as examples. As the parallel_compute() function in pyUSID does not scale up to multi-node machines, Dask_jobqueue can be used to automate computation to clusters and supercomputers. This tutorial uses Summit at Oak Ridge National Laboratory, but can be applied to HPC systems that use LFS files to submit and deploy jobs. 

It is strongly recommended that Jupyter Lab is used on the remote machine to facilitate the interactive session.

**Note**: The contents of this tutorial can be run in an interactive session on Summit. Please open this Jupyter Lab from a Summit login node.

## Jupyter Lab on Summit 
In order to properly complete and run this tutorial, follow these instructions to open Jupyter Lab on the remote machine, Summit.
***
**Open this Jupyter Notebook using Jupyter Lab on a Summit login node.**

Note: If the repo containing this Jupyter Notebook does not exist in your current Summit path on the login node, it will automatically download. If it does, it will make sure the repo is up-to-date.
***
1. On summit login node, run this script and leave terminal running:
> `source resources/jupyterlab.sh`

2. Open a new terminal and run this on your local machine:
> `ssh -L 8887:localhost:8887 user@login##.summit.olcf.ornl.gov`

3. Now, open `localhost:8887` on your local browser and navigate to this Jupyter Notebook.
***
Now, let us begin the tutorial.

Import all necessary modules

In [ ]:
from dask_jobqueue import LFSCluster
from dask.distributed import Client, LocalCluster
import pyUSID as usid


In the following format 
> `parameter: default     # Option flag and description`
    
    name: dask-worker           # -N option, name of job.

***Dask worker options***
    
    cores: null                 # Total number of cores per job.
    memory: null                # Total amount of memory per job.
    processes: 1                # Number of Python processes per job

    interface: null             # Network interface to use like eth0 or ib0
    death-timeout: 60           # Number of seconds to wait if a worker can not find a scheduler
    local-directory: null       # Fast local storage location (/scratch, $TMPDIR)

***LFS resource manager options***
    
    shebang: "#!/usr/bin/env bash"    # Command language
    queue: null                       # -q option
    project: null                     # -A option
    walltime: '00:30:00'              # -l walltime: xx:xx:xx
    extra: []
    env-extra: []
    resource-spec: null
    job-extra: []
    log-directory: null

In order to accomodate an interactive session, let us add a few more commands. Create a list of additional commands then put it in job_extra parameter.

In [ ]:
job_extra = list[]
# Command to begin interactive session, which we will open in terminal later.
job_extra.append('-Is')
# The LDAP group list, cades-birthright in this case.
job_extra.append('-W group_list=cades-birthright')

Set up nodes and LFS script for requesting then running a job on Summit.

In [ ]:
local_directory = '/.' # Current directory, feel free to change.

cluster = PBSCluster(name='HelloWorld',
                     project='birthright',
                     resource_spec='nodes=2:ppn=32:qos=std',
                     walltime='02:00:00',
                     local_directory=local_directory)

## Dask-jobqueue Features
Now, let us mess around with some of the features of dask-jobqueue.
***


In [ ]:
cluster.adapt(minimum=18, maximum=360) # Scale between 18 and 360 workers